In [62]:
from openai import OpenAI
import os
import base64
import requests
import json
import requests
import pandas as pd

import time
from http.client import RemoteDisconnected

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef

In [63]:
path = 'C:\\Users\\User\\Desktop\\Data\\SCAI\\Image_analysis_AI\\'
path_1 = 'C:\\Users\\User\\Desktop\\Data\\SCAI\\'

In [109]:
def gpt_authenticate():
    """
    Authenticates with the OpenAI GPT API using a predefined API key and returns the authenticated client object.
    
    Returns:
        client (OpenAI): Authenticated OpenAI API client.
    """
    API_key = "GPT_api_key"
    client = OpenAI(api_key = API_key)
    models = client.models.list()
    return client


def check_image_url(url, retries=3):
    """
    Checks the accessibility of a given image URL by sending an HTTP GET request and handling retries on failure.
    It validates whether the URL points to an image by examining the Content-Type header.

    Args:
        url (str): The URL of the image to be checked.
        retries (int, optional): Number of retry attempts in case of connection failures. Default is 3.

    Returns:
        bool: Returns False if the URL is accessible and contains an image; True if an error occurred or the content is not an image.
    """
    attempt = 0

    while attempt < retries:
        try:
            # Send a GET request to the image URL
            response = requests.get(url, timeout=5)  # Set a timeout to avoid hanging
            
            # Check if the response status code is 200 (OK)
            if response.status_code == 200:
                # Get the Content-Type header from the response
                content_type = response.headers.get('Content-Type', '')

                # Check if the Content-Type is an image format
                if 'image' in content_type:
                    print(f"Success: URL {url} is accessible and is of type {content_type}.")
                    return False  # No error
                else:
                    print(f"Error: URL {url} is not an image. Content-Type: {content_type}")
                    return True  # Error occurred
            else:
                print(f"Error: URL {url} returned status code {response.status_code}.")
                return True  # Error occurred

        except (requests.ConnectionError, requests.Timeout, RemoteDisconnected) as e:
            attempt += 1
            print(f"Attempt {attempt}/{retries} failed for {url}: {e}")
            if attempt < retries:
                print("Retrying after a short delay...")
                time.sleep(2)  # Wait for 2 seconds before retrying
            else:
                print(f"Failed to access URL after {retries} attempts: {url}")
                return True  # Error occurred after retries
        except Exception as e:
            print(f"Error processing {url}: {e}")
            return True  # Error occurred due to unknown exception

    return True  # Fallback in case of any unexpected behavior


def URLs_access_check(dataframe, retries=3):
    """
    Iterates over a DataFrame containing image URLs, checks the accessibility of each URL, and stores the results.
    
    Args:
        dataframe (pandas.DataFrame): DataFrame containing a column 'image_url' with image URLs.
        retries (int, optional): Number of retry attempts in case of connection failures. Default is 3.

    Returns:
        pandas.DataFrame: DataFrame containing two columns: 'URL' and 'Access' (Yes/No), indicating the accessibility of each URL.
    """
    # Initialize list to store the result
    access_data = []

    # Iterate through each URL in the dataframe to check accessibility
    for idx, row in dataframe.iterrows():
        url = row['image_url']

        # Check if the image URL is accessible with retry mechanism
        access = "No" if check_image_url(url, retries=retries) else "Yes"

        # Append the result (URL, Access) to the list
        access_data.append([url, access])

    # Create a new DataFrame to store the results
    access_df = pd.DataFrame(access_data, columns=['URL', 'Access'])

    return access_df


def gpt_extract(client, url):
    """
    Uses GPT to analyze an image and determine its relevance based on predefined criteria. 
    The URL is passed to the GPT model, which returns either "Yes" or "No" indicating the relevance of the image.

    Args:
        client (OpenAI): The authenticated OpenAI API client.
        url (str): The URL of the image to be analyzed.

    Returns:
        str: The GPT's assessment of the image's relevance ("Yes" or "No").
    """
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": '''Image URL is given. Analyze this image and assign relevance to it: "Not" for irrelevant images and "Yes" for relevant ones.
                                      Follow this classification:
                                      Relevant:
                                      Images which:
                                      Сlearly demonstrate relationship between Covid-19 and neurodegeneration (any neurological impacts).
                                      Don't contain lots of text (no more than 500 characters).
                                      Don't just depict a research outline.
                                      Don't be just graphs or represent photos derived from scientific tools (microscopic, histological images) and data visualization.
                                      Are likely to be cartoons drawn by article authors.
                                      
                                      Irrelevant:
                                      Unrelated images, for example just an image of a virus particle or a sick person.
                                      Images where correct interpretation of the data is impossible.
                                      Images which display insights into Covid-19 OR Neurodegeneration, if one is present and the other is missing.
                                      
                                      Your answer should contain only a final decision in the following format: No/Yes (without dots)
                                      Don't write anything else!'''},
           {
            "type": "image_url",
            "image_url": {
              "url": url,
            },
          },
        ],
      }
    ],
    max_tokens=900,
  )
    content = response.choices[0].message.content
    return content


def get_GPT_answers(dataframe):
    """
    Authenticates with GPT, processes a DataFrame containing image URLs, and retrieves the relevance classification for each image.

    Args:
        dataframe (pandas.DataFrame): DataFrame containing a column 'image_url' with image URLs.

    Returns:
        list: A list of lists where each inner list contains the image URL and the GPT's relevance classification or "Error" in case of failure.
    """
    client = gpt_authenticate()

    # Initialize an empty list to store parsed data
    parsed_data = []

    # Loop through each row in the dataframe
    for idx, row in dataframe.iterrows():
        try:
            url = row["image_url"]  # Extract image URL

            # Extract content from the image using GPT
            relev_check = gpt_extract(client, url)

            print(url, relev_check)

    # WRITING TO A TABLE

            parsed_data.append([url, relev_check])


        except Exception as e:
            # Print the error and continue to the next row
            print(f"Error processing {idx, row}: {e}")
            parsed_data.append([url, "Error"])
            continue  # Skip to the next row in case of an error
    
    return parsed_data

def relevance_check_main(path, name_raw_data):
    # Convert the Excel file into dataframe
    data_raw = pd.read_excel(path + name_raw_data)

    # Drop 'Unnamed: 0' only if it exists in the DataFrame
    if 'Unnamed: 0' in data_raw.columns:
        data_raw.drop('Unnamed: 0', axis=1, inplace=True)
    
    # Get relevance data
    parsed_data_all = get_GPT_answers(data_raw)
    
    # Loop through the list and clean up 'Yes.' or 'No.' to 'Yes' or 'No'
    for item in parsed_data_all:
        # Check if the second element ends with a period and is 'Yes.' or 'No.'
        if item[1] in ['Yes.', 'No.']:
            item[1] = item[1].rstrip('.')
            
    # Create a new DataFrame to store parsed results
    relevance_GPT_all = pd.DataFrame(parsed_data_all, columns=['URL', 'Relevance_GPT'])
    relevance_GPT_all.to_excel(path+"Relevance_assignment_GPT_4o.xlsx", index=False)
    
    # Store only relevant URLs
    relevant_URLs = relevance_GPT_all[relevance_GPT_all['Relevance_GPT'] == 'Yes']
    relevant_URLs.reset_index(drop=True, inplace=True)
    relevant_URLs.to_excel(path+"Relevant_URLs_only_GPT_4o.xlsx", index=False)
    
    # Number of images which GPT can't process
    print('Initial total number of URLs:', len(data_raw))
    print("Number of images which GPT can't process",len(relevance_GPT_all.loc[relevance_GPT_all['Relevance_GPT'] == 'Error']))
    print('Number of relevant images (GPT):', relevant_URLs['Relevance_GPT'].value_counts().get('Yes', 0))
    

def consistency_evaluation(GPT_and_manual_sampled_df):
    """
    Evaluates the consistency between GPT and manual relevance classifications using various metrics such as accuracy, precision, recall, F1-score, and MCC. 
    Also generates a confusion matrix and calculates specificity.

    Args:
        GPT_and_manual_sampled_df (pandas.DataFrame): DataFrame containing 'Relevance_GPT' and 'Relevance_manual' columns.

    Returns:
        None: Prints out the evaluation metrics and confusion matrix.
    """
    # Count inconsistencies where Relevance_GPT is not equal to Relevance_manual
    inconsistencies = (GPT_and_manual_sampled_df['Relevance_GPT'] != GPT_and_manual_sampled_df['Relevance_manual']).sum()
    print(f'Total inconsistencies: {inconsistencies}')

    # Accuracy
    accuracy = (GPT_and_manual_sampled_df['Relevance_GPT'] == GPT_and_manual_sampled_df['Relevance_manual']).mean()
    print(f'Accuracy: {accuracy:.2f}')

    # Precision
    precision = precision_score(GPT_and_manual_sampled_df['Relevance_manual'], GPT_and_manual_sampled_df['Relevance_GPT'], pos_label='Yes')
    print(f'Precision: {precision:.2f}')

    # Recall
    recall = recall_score(GPT_and_manual_sampled_df['Relevance_manual'], GPT_and_manual_sampled_df['Relevance_GPT'], pos_label='Yes')
    print(f'Recall: {recall:.2f}')

    # F1-Score
    f1 = f1_score(GPT_and_manual_sampled_df['Relevance_manual'], GPT_and_manual_sampled_df['Relevance_GPT'], pos_label='Yes')
    print(f'F1-Score: {f1:.2f}')

    # Confusion Matrix
    cm = confusion_matrix(GPT_and_manual_sampled_df['Relevance_manual'], GPT_and_manual_sampled_df['Relevance_GPT'], labels=['Yes', 'No'])
    print('Confusion Matrix:')
    print(cm)

    # Specificity (True Negative Rate)
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / (tn + fp)
    print(f'Specificity: {specificity:.2f}')

    # MCC
    mcc = matthews_corrcoef(GPT_and_manual_sampled_df['Relevance_manual'], GPT_and_manual_sampled_df['Relevance_GPT'])
    print(f'MCC: {mcc:.2f}')


## Open the raw file

In [10]:
file_data = path+ "enrichment_cleaned.xlsx"
data_raw = pd.read_excel(file_data)
data_raw.drop('Unnamed: 0', axis=1, inplace=True)
data_raw

,thumbnail_url,image_url
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",https://media.springernature.com/lw1200/spring...
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",https://www.researchgate.net/publication/36170...
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",https://www.frontiersin.org/files/Articles/103...
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",https://www.frontiersin.org/files/Articles/583...
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...",https://media.springernature.com/lw1200/spring...
...,...,...
6314,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.frontiersin.org/files/MyHome%20Art...
6315,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.researchgate.net/publication/23227...
6316,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.frontiersin.org/files/Articles/402...
6317,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.researchgate.net/profile/Teresa-Ni...


## The main GPT-4o relevance analysis (whole image pull)

In [76]:
parsed_data_all = get_GPT_answers(data_raw)

SyncPage[Model](data=[Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal'), Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system'), Model(id='chatgpt-4o-latest', created=1723515131, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview', created=1727460443, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview-2024-10-01', created=1727389042, object='model', owned_by='system'), Model(id='gpt-4-0125-previe

Error processing (17, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/37263...
Name: 17, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/372634934/figure/fig1/AS:11431281177211368@1690419802669/The-SARS-CoV-2-virus-may-cause-brain-pathology-through-both-direct-and-indirect.png.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
Error processing (18, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/33950...
Name: 18, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/339500154/figure/fig3/AS:11431281249859700@1717692151745/The-plausible-hypothesis-for-selective-neurodegeneration-in-various-neurodegenerative_Q320.jpg.', 'type': 'invalid_request_error', 'param': None,

Error processing (49, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://ars.els-cdn.com/content/image/1-s2.0-S...
Name: 49, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://ars.els-cdn.com/content/image/1-s2.0-S1471490624000644-gr3.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://www.frontiersin.org/files/Articles/1011936/fmed-10-1011936-HTML-r1/image_m/fmed-10-1011936-g001.jpg No
Error processing (51, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/34465...
Name: 51, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/344657692/figure/fig1/AS:946598372712450@1602698047940/Central-and-peripheral-inflammatory-immune-response-in-neurodegeneration-Upon-injury.png.', 'type': 'invalid_request_error', 'param': None, 'code':

Error processing (76, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/34887...
Name: 76, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/348878411/figure/fig1/AS:11431281254351669@1719155840409/Mechanisms-of-virion-attachment-infection-and-possibles-routes-of-SARS-CoV-2_Q320.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
Error processing (77, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/35917...
Name: 77, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/359175187/figure/fig1/AS:1139011905171457@1648573008629/Transmission-and-life-cycle-of-SARS-CoV-2-causing-COVID-19-SARS-CoV-2-is-transmitted-via_Q320.jpg.', 'type': 'invalid_request_error', 'param': N

https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs40035-023-00337-1/MediaObjects/40035_2023_337_Fig2_HTML.png Yes
https://www.altex.org/public/journals/1/submission_2367_2429_coverImage_en_US.png Yes
Error processing (105, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://ars.els-cdn.com/content/image/1-s2.0-S...
Name: 105, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://ars.els-cdn.com/content/image/1-s2.0-S2666354621002027-gr3.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11011-021-00800-w/MediaObjects/11011_2021_800_Fig2_HTML.png No
https://www.frontiersin.org/files/Articles/456680/fcimb-09-00138-HTML/image_m/fcimb-09-00138-g001.jpg No
Error processing (108, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.resear

...

https://media.springernature.com/lw1200/springer-static/image/art%3A10.1186%2Fs13024-021-00465-0/MediaObjects/13024_2021_465_Fig5_HTML.png No
Error processing (6193, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.biorxiv.org/content/biorxiv/early/...
Name: 6193, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.biorxiv.org/content/biorxiv/early/2023/11/07/2023.11.06.565824/F8.large.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
Error processing (6194, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://pubs.acs.org/cms/10.1021/acsomega.0c05...
Name: 6194, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://pubs.acs.org/cms/10.1021/acsomega.0c05638/asset/images/acsomega.0c05638.social.jpeg_v03.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
Error proc

https://cdn.mdedge.com/files/s3fs-public/146030_graphic_web.png No
https://journals.plos.org/plosone/article/figure/image?size=medium&id=10.1371/journal.pone.0247248.g002 No
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1038%2Fs41467-022-34068-0/MediaObjects/41467_2022_34068_Fig5_HTML.png No
Error processing (6220, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://febs.onlinelibrary.wiley.com/cms/asset...
Name: 6220, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://febs.onlinelibrary.wiley.com/cms/asset/a1e56eaa-21e8-4b22-8c2c-f0e9ee5df5e2/feb212426-fig-0002-m.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://journals.plos.org/plosgenetics/article/figure/image?size=medium&id=10.1371/journal.pgen.1009994.g005 No
Error processing (6222, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchg

https://media.springernature.com/m685/springer-static/image/art%3A10.1038%2Fs41392-023-01378-8/MediaObjects/41392_2023_1378_Fig1_HTML.png No
Error processing (6242, thumbnail_url    data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...
image_url        https://www.researchgate.net/publication/31425...
Name: 6242, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/314256709/figure/fig1/AS:498280883605504@1495810828169/Most-common-aging-and-age-related-human-diseases.png.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://media.springernature.com/lw685/springer-static/image/art%3A10.1038%2Fs41392-022-01251-0/MediaObjects/41392_2022_1251_Fig8_HTML.png No
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs10522-021-09910-5/MediaObjects/10522_2021_9910_Fig4_HTML.png No
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1186%2Fs12964-024-01663

https://pub.mdpi-res.com/cimb/cimb-46-00287/article_deploy/html/images/cimb-46-00287-g001-550.jpg?1715766039 No
Error processing (6277, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/33595...
Name: 6277, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/335954216/figure/fig1/AS:11431281181657403@1692117805057/The-conceptual-framework-of-Geroscience-hypothesis-and-goals-linkage-between-aging-and.png.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://europepmc.org/articles/PMC5991498/bin/nihms858344f3.jpg No
https://scx2.b-cdn.net/gfx/news/hires/2020/agerelatedim.jpg No
https://upload.wikimedia.org/wikipedia/en/2/25/Aging_Disease_2015_cover.jpg No
Error processing (6281, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/34627...


Error processing (6306, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/publication/33801...
Name: 6306, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/publication/338013854/figure/fig1/AS:11431281255827289@1719438701825/Conceptualization-of-the-contribution-of-aging-as-a-physiological-process-Biological_Q320.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://longevityfacts.com/wp-content/uploads/2017/10/The-Hallmarks-of-Aging-gr6_lrg.-%C2%A9-2013-Elsevier-Inc.jpg No
Error processing (6308, thumbnail_url    https://encrypted-tbn0.gstatic.com/images?q=tb...
image_url        https://www.researchgate.net/profile/Christoph...
Name: 6308, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://www.researchgate.net/profile/Christopher-Yukna/post/Are_the_Diseases_of_Civilization_autoimmune_or_

In [79]:
# Create a new DataFrame to store parsed results

# Loop through the list and clean up 'Yes.' or 'No.' to 'Yes' or 'No'
for item in parsed_data_all:
    # Check if the second element ends with a period and is 'Yes.' or 'No.'
    if item[1] in ['Yes.', 'No.']:
        item[1] = item[1].rstrip('.')

relevance_GPT_all = pd.DataFrame(parsed_data_all, columns=['URL', 'Relevance_GPT'])
#relevance_GPT_all.to_excel(r"Relevance_GPT_analysis.xlsx", index=False)
relevance_GPT_all

,URL,Relevance_GPT
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.researchgate.net/publication/36170...,Error
2,https://www.frontiersin.org/files/Articles/103...,Yes
3,https://www.frontiersin.org/files/Articles/583...,Yes
4,https://media.springernature.com/lw1200/spring...,Yes
...,...,...
6314,https://www.frontiersin.org/files/MyHome%20Art...,No
6315,https://www.researchgate.net/publication/23227...,Error
6316,https://www.frontiersin.org/files/Articles/402...,No
6317,https://www.researchgate.net/profile/Teresa-Ni...,Error


## URLs access check

In [11]:
access_df_all = URLs_access_check(data_raw)
access_df_all

Success: URL https://media.springernature.com/lw1200/springer-static/image/art%3A10.1186%2Fs13062-020-00282-3/MediaObjects/13062_2020_282_Fig1_HTML.png is accessible and is of type image/png.
Error: URL https://www.researchgate.net/publication/361708170/figure/fig1/AS:11431281105587509@1670463859797/Activation-of-systemic-inflammation-and-potential-routes-of-severe-acute-respiratory_Q320.jpg returned status code 403.
Success: URL https://www.frontiersin.org/files/Articles/1039427/fimmu-13-1039427-HTML/image_m/fimmu-13-1039427-g001.jpg is accessible and is of type image/jpeg.
Success: URL https://www.frontiersin.org/files/Articles/583459/fneur-11-583459-HTML/image_m/fneur-11-583459-g001.jpg is accessible and is of type image/jpeg.
Success: URL https://media.springernature.com/lw1200/springer-static/image/art%3A10.1038%2Fs41392-021-00818-7/MediaObjects/41392_2021_818_Fig4_HTML.png is accessible and is of type image/png.
Error: URL https://www.researchgate.net/publication/382969249/figure

Success: URL https://pub.mdpi-res.com/cells/cells-12-01763/article_deploy/html/images/cells-12-01763-g001-550.jpg?1688200925 is accessible and is of type image/jpeg.
Success: URL https://www.frontiersin.org/files/Articles/1020092/fnagi-14-1020092-HTML/image_m/fnagi-14-1020092-g003.jpg is accessible and is of type image/jpeg.
Error: URL https://www.researchgate.net/publication/371440897/figure/fig3/AS:11431281166673693@1686329744679/sualises-the-content-of-31-and-32-Created-with-BioRendercom_Q320.jpg returned status code 403.
Success: URL https://ars.els-cdn.com/content/image/1-s2.0-S1471490624000644-gr3.jpg is accessible and is of type image/jpeg.
Success: URL https://www.frontiersin.org/files/Articles/1011936/fmed-10-1011936-HTML-r1/image_m/fmed-10-1011936-g001.jpg is accessible and is of type image/jpeg.
Error: URL https://www.researchgate.net/publication/344657692/figure/fig1/AS:946598372712450@1602698047940/Central-and-peripheral-inflammatory-immune-response-in-neurodegeneration-Up

Success: URL https://ars.els-cdn.com/content/image/1-s2.0-S0024320522006816-gr2.jpg is accessible and is of type image/jpeg.
Success: URL https://media.springernature.com/w215h120/springer-static/image/art%3A10.1007%2Fs00415-020-10131-y/MediaObjects/415_2020_10131_Fig1_HTML.png is accessible and is of type image/png.
Error: URL https://www.researchgate.net/publication/363149966/figure/fig5/AS:11431281085563724@1663785379915/Schematic-illustration-of-how-global-lockdown-and-SARS-CoV-2-lead-to-neuropsychiatric_Q320.jpg returned status code 403.
Error: URL https://www.researchgate.net/publication/358012876/figure/fig1/AS:1114931374104576@1642831762992/Neurodegenerative-diseases-and-the-types-of-neurons-affected.png returned status code 403.
Success: URL https://images-provider.frontiersin.org/api/ipx/w=1200&f=png/https://www.frontiersin.org/files/Articles/1185671/fnagi-15-1185671-HTML/image_m/fnagi-15-1185671-g001.jpg is accessible and is of type image/png.
Error: URL https://journals.sag

Error: URL https://www.researchgate.net/publication/378851517/figure/fig1/AS:11431281228512555@1710039789674/The-hypothetical-role-of-SARS-CoV-2-in-mitochondrial-damage-and-in-Parkinsons-disease_Q320.jpg returned status code 403.
Error: URL https://www.tandfonline.com/cms/asset/755928ae-0594-4acd-8249-ae7e7ec92cb6/ines_a_2056460_f0002_c.jpg returned status code 403.
Error: URL https://www.researchgate.net/publication/345771104/figure/fig1/AS:983191221460992@1611422462037/Mechanisms-of-coronaviral-infection-of-the-brain-SARS-CoV-2-can-enter-the-nervous-system.png returned status code 403.
Success: URL https://benthamsciencepublishers.wordpress.com/wp-content/uploads/2022/11/big-013.jpg?w=1024 is accessible and is of type image/jpeg.
Success: URL https://www.frontiersin.org/files/Articles/568710/fneur-11-01044-HTML/image_m/fneur-11-01044-g001.jpg is accessible and is of type image/jpeg.
Success: URL https://insidescientific.com/wp-content/uploads/2022/10/fx1_lrg.jpg is accessible and is 

Error: URL https://www.researchgate.net/publication/352038969/figure/fig3/AS:1030578837147648@1622720550623/Conformational-changes-of-SARS-CoV-2-spike-protein.png returned status code 403.
Error: URL https://www.researchgate.net/publication/358539733/figure/fig3/AS:11431281181922989@1692211623228/Recommendations-to-manage-Omicron-SARS-CoV-2-variant-Experts-recommend-managing-Omicron_Q320.jpg returned status code 403.
Success: URL https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs41983-023-00705-8/MediaObjects/41983_2023_705_Fig1_HTML.png is accessible and is of type image/png.
Success: URL https://www.vai.org/wp-content/uploads/2022/06/Brundin-P-et-al.-Trends-in-Neurosciences-01.png is accessible and is of type image/png.
Success: URL https://ars.els-cdn.com/content/image/1-s2.0-S0167732222003129-ga1.jpg is accessible and is of type image/jpeg.
Success: URL https://www.frontiersin.org/files/Articles/662786/fnagi-13-662786-HTML/image_m/fnagi-13-662786-g001.jp

...

Error: URL https://www.researchgate.net/profile/Daniel-Correa-9/publication/343337408/figure/tbl2/AS:951232692776962@1603802955739/Seven-types-of-neuroscience-ward-beds-at-Montefiore-Medical-Center_Q320.jpg returned status code 403.
Error: URL https://onlinelibrary.wiley.com/cms/asset/adfa977b-4856-4fe5-a38c-d950954b2dba/ana.v88.2.cover.jpg?trick=1714633179995 returned status code 403.
Error: URL https://www.researchgate.net/profile/Ahmed-Elkady-17/publication/345319376/figure/tbl1/AS:954424780402688@1604564008110/Demographic-Information-of-Study-Participants_Q320.jpg returned status code 403.
Error: URL https://www.researchgate.net/publication/5394545/figure/fig2/AS:667197357694976@1536083654060/High-levels-of-immunostaining-for-amyloid-b-Tau-and-OX-6-in-the-cortex-and-hippocampus.png returned status code 403.
Success: URL https://www.biorxiv.org/content/biorxiv/early/2023/11/07/2023.11.06.565824/F3.large.jpg is accessible and is of type image/jpeg.
Success: URL https://media.springer

Error: URL https://www.researchgate.net/profile/Mahmud-Reaz-2/publication/351561600/figure/fig2/AS:1023042230689793@1620923683922/Time-to-recovery-in-the-treatment-and-control-groups-with-and-without-censored-data_Q320.jpg returned status code 403.
Error: URL https://www.researchgate.net/profile/Ali-Afrasiabi-3/publication/357507579/figure/tbl1/AS:1107597952327685@1641083338392/Acute-ischemic-stroke-AIS-Covid-19_Q320.jpg returned status code 403.
Error: URL https://www.researchgate.net/publication/263093206/figure/fig1/AS:296063115251714@1447598355146/Inhibition-by-heparin-of-P-falciparum-in-vitro-growth-Heparin-was-added-to-RBC-pRBC.png returned status code 403.
Success: URL https://cdn.mdedge.com/files/s3fs-public/146030_graphic_web.png is accessible and is of type image/png.
Success: URL https://journals.plos.org/plosone/article/figure/image?size=medium&id=10.1371/journal.pone.0247248.g002 is accessible and is of type image/png.
Success: URL https://media.springernature.com/lw1200/s

Success: URL https://dm5migu4zj3pb.cloudfront.net/manuscripts/169000/169922/medium/JCI169922.f1.jpg is accessible and is of type image/jpeg.
Success: URL https://erj.ersjournals.com/content/erj/44/5/1332/F1.large.jpg?width=800&height=600&carousel=1 is accessible and is of type image/jpeg.
Success: URL https://www.aging-us.com/article/202136/figure/f1/large is accessible and is of type image/png.
Success: URL https://cdn.sci.news/images/2013/03/image_942.jpg is accessible and is of type image/jpeg.
Success: URL https://images.squarespace-cdn.com/content/v1/6282a1a906845279cb747027/f4d0ec3e-c8a2-4cf4-9af0-f4259ae99da3/Geroscience+hypothesis+v1.2.png is accessible and is of type image/png.
Success: URL https://ars.els-cdn.com/content/image/1-s2.0-S2212877823000893-gr1.jpg is accessible and is of type image/jpeg.
Success: URL https://media.springernature.com/lw1200/springer-static/image/art%3A10.1038%2Fs41392-022-01251-0/MediaObjects/41392_2022_1251_Fig7_HTML.png is accessible and is of ty

Error: URL https://www.researchgate.net/publication/338013854/figure/fig1/AS:11431281255827289@1719438701825/Conceptualization-of-the-contribution-of-aging-as-a-physiological-process-Biological_Q320.jpg returned status code 403.
Success: URL https://longevityfacts.com/wp-content/uploads/2017/10/The-Hallmarks-of-Aging-gr6_lrg.-%C2%A9-2013-Elsevier-Inc.jpg is accessible and is of type image/jpeg.
Error: URL https://www.researchgate.net/profile/Christopher-Yukna/post/Are_the_Diseases_of_Civilization_autoimmune_or_a_manifestation_of_a_leaky_gut/attachment/656b3d36ad82d637bf12f168/AS%3A11431281208936430%401701526838284/download/antioxidants-11-00865-g001.png returned status code 403.
Success: URL https://ars.els-cdn.com/content/image/1-s2.0-S1876034120304986-gr1.jpg is accessible and is of type image/jpeg.
Error: URL https://www.researchgate.net/publication/361623136/figure/fig1/AS:1175879564705792@1657362943222/Factors-affecting-healthy-aging.png returned status code 403.
Success: URL http

,URL,Access
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.researchgate.net/publication/36170...,No
2,https://www.frontiersin.org/files/Articles/103...,Yes
3,https://www.frontiersin.org/files/Articles/583...,Yes
4,https://media.springernature.com/lw1200/spring...,Yes
...,...,...
6314,https://www.frontiersin.org/files/MyHome%20Art...,Yes
6315,https://www.researchgate.net/publication/23227...,No
6316,https://www.frontiersin.org/files/Articles/402...,Yes
6317,https://www.researchgate.net/profile/Teresa-Ni...,No


In [16]:
# Saving file
#access_df_all.to_excel(r"URL_access_GPT_analysis.xlsx", index=False)

In [17]:
access_df_all = pd.read_excel(path_1+"URL_access_GPT_analysis.xlsx")
access_df_all

,URL,Access
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.researchgate.net/publication/36170...,No
2,https://www.frontiersin.org/files/Articles/103...,Yes
3,https://www.frontiersin.org/files/Articles/583...,Yes
4,https://media.springernature.com/lw1200/spring...,Yes
...,...,...
6314,https://www.frontiersin.org/files/MyHome%20Art...,Yes
6315,https://www.researchgate.net/publication/23227...,No
6316,https://www.frontiersin.org/files/Articles/402...,Yes
6317,https://www.researchgate.net/profile/Teresa-Ni...,No


In [20]:
# Number of images with no access
print('Number of images with no access',len(access_df_all.loc[access_df_all['Access'] == 'No']))
# Number of images which GPT can't process
print("Number of images which GPT can't process",len(relevance_GPT_all.loc[relevance_GPT_all['Relevance_GPT'] == 'Error']))

Number of images with no access 2254
Number of images which GPT can't process 2705


## Open GPT-4o relevance analysis table to futher processing

In [21]:
relevance_GPT_all = pd.read_excel(path_1+"Relevance_GPT_analysis.xlsx")
relevance_GPT_all

,URL,Relevance_GPT
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.researchgate.net/publication/36170...,Error
2,https://www.frontiersin.org/files/Articles/103...,Yes
3,https://www.frontiersin.org/files/Articles/583...,Yes
4,https://media.springernature.com/lw1200/spring...,Yes
...,...,...
6314,https://www.frontiersin.org/files/MyHome%20Art...,No
6315,https://www.researchgate.net/publication/23227...,Error
6316,https://www.frontiersin.org/files/Articles/402...,No
6317,https://www.researchgate.net/profile/Teresa-Ni...,Error


In [46]:
relevance_GPT_success = relevance_GPT_all.drop(relevance_GPT_all[relevance_GPT_all.Relevance_GPT == "Error"].index)
relevance_GPT_success = relevance_GPT_success.reset_index(drop=True)
print('Successfully analyzed URLs:',len(relevance_GPT_success))
print('Number of relevant images (GPT):', relevance_GPT_success['Relevance_GPT'].value_counts().get('Yes', 0))

Successfully analyzed URLs: 3614
Number of relevant images (GPT): 626


## Sample 500 URLs from successfully analyzed by GPT for manual analysis

In [43]:
# Randomly sample 500 rows from the DataFrame
sampled_data = relevance_GPT_success.sample(n=500, random_state=42)  # random_state ensures reproducibility
sampled_data.reset_index(drop=True, inplace=True)
# Save sampled data alalyzed by GPT
sampled_data.to_excel("C:\\Users\\User\\Desktop\\Data\\SCAI\\Image_analysis_AI\\GPT_analyzed_sampled.xlsx")

# Save just sampled URLs for manual assessment
(sampled_data[['URL']]).to_excel('C:\\Users\\User\\Desktop\\Data\\SCAI\\Image_analysis_AI\\URL_sampled_list.xlsx', index=False)
sampled_data

,URL,Relevance_GPT
0,https://www.medrxiv.org/content/medrxiv/early/...,No
1,https://media.springernature.com/m685/springer...,No
2,https://assets.technologynetworks.com/producti...,No
3,https://media.springernature.com/lw685/springe...,No
4,https://image.slidesharecdn.com/neurodegenerat...,No
...,...,...
495,https://www.mdpi.com/ijms/ijms-25-02268/articl...,Yes
496,https://media.springernature.com/lw685/springe...,No
497,https://cdn.images.express.co.uk/img/dynamic/1...,No
498,https://geripal.org/wp-content/uploads/2019/10...,No


## Comparison of GPT-4o relevance assignment with manual

In [37]:
# Read manually analysed URLs table
sampled_manual = pd.read_excel(path+"URL_sampled_manual.xlsx")
sampled_manual

,URL,Relevance_manual
0,https://www.medrxiv.org/content/medrxiv/early/...,No
1,https://media.springernature.com/m685/springer...,No
2,https://assets.technologynetworks.com/producti...,No
3,https://media.springernature.com/lw685/springe...,No
4,https://image.slidesharecdn.com/neurodegenerat...,No
...,...,...
495,https://www.mdpi.com/ijms/ijms-25-02268/articl...,No
496,https://media.springernature.com/lw685/springe...,No
497,https://cdn.images.express.co.uk/img/dynamic/1...,No
498,https://geripal.org/wp-content/uploads/2019/10...,No


In [49]:
comparison_sample = sampled_data
comparison_sample['Relevance_manual'] = sampled_manual['Relevance_manual']
#comparison_sample.to_excel("C:\\Users\\User\\Desktop\\Data\\SCAI\\Image_analysis_AI\\Comparison_GPT_Manual_relevance.xlsx")
comparison_sample

,URL,Relevance_GPT,Relevance_manual
0,https://www.medrxiv.org/content/medrxiv/early/...,No,No
1,https://media.springernature.com/m685/springer...,No,No
2,https://assets.technologynetworks.com/producti...,No,No
3,https://media.springernature.com/lw685/springe...,No,No
4,https://image.slidesharecdn.com/neurodegenerat...,No,No
...,...,...,...
495,https://www.mdpi.com/ijms/ijms-25-02268/articl...,Yes,No
496,https://media.springernature.com/lw685/springe...,No,No
497,https://cdn.images.express.co.uk/img/dynamic/1...,No,No
498,https://geripal.org/wp-content/uploads/2019/10...,No,No


## Extended consistensy evaluation

In [110]:
consistency_evaluation(comparison_sample)

Total inconsistencies: 49
Accuracy: 0.90
Precision: 0.48
Recall: 0.96
F1-Score: 0.64
Confusion Matrix:
[[ 44   2]
 [ 47 407]]
Specificity: 0.96
MCC: 0.64


The precision of 0.48 means that when the model predicts a positive class (Irrelevant Images), it is correct 48% of the time. A relatively low precision suggests that the model is making a significant number of false-positive (Marks Irrelevant as Relevant) predictions.

True Positives (TP) = 407: These are cases where the model correctly predicted Irrelevant Images (No).

True Negatives (TN) = 44: These are cases where the model correctly predicted Relevant Images (Yes).

False Positives (FP) = 2: These are cases where the model incorrectly predicted the Irrelevant Images (No) (predicted Irrelevant Images, but it was actually Relevant Images).

False Negatives (FN) = 47: These are cases where the model incorrectly predicted the Relevant Images (Yes) (predicted Relevant Images, but it was actually Irrelevant Images).

## Further purification attempt: 2nd run

In [80]:
Relevant_URLs_raw = relevance_GPT_success[relevance_GPT_success['Relevance_GPT'] == 'Yes']
Relevant_URLs_raw = Relevant_URLs_raw.rename(columns={'URL': 'image_url'})
Relevant_URLs_raw.reset_index(drop=True, inplace=True)
Relevant_URLs_raw

,image_url,Relevance_GPT
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.frontiersin.org/files/Articles/103...,Yes
2,https://www.frontiersin.org/files/Articles/583...,Yes
3,https://media.springernature.com/lw1200/spring...,Yes
4,https://media.springernature.com/w215h120/spri...,Yes
...,...,...
621,https://i.ytimg.com/vi/fs3wDh_7pPE/hq720.jpg?s...,Yes
622,https://media.springernature.com/lw685/springe...,Yes
623,https://media.springernature.com/lw685/springe...,Yes
624,https://www.aging-us.com/article/202136/figure...,Yes


In [81]:
parsed_data_2nd = get_GPT_answers(Relevant_URLs_raw)

https://media.springernature.com/lw1200/springer-static/image/art%3A10.1186%2Fs13062-020-00282-3/MediaObjects/13062_2020_282_Fig1_HTML.png Yes
https://www.frontiersin.org/files/Articles/1039427/fimmu-13-1039427-HTML/image_m/fimmu-13-1039427-g001.jpg Yes
https://www.frontiersin.org/files/Articles/583459/fneur-11-583459-HTML/image_m/fneur-11-583459-g001.jpg Yes
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1038%2Fs41392-021-00818-7/MediaObjects/41392_2021_818_Fig4_HTML.png Yes
https://media.springernature.com/w215h120/springer-static/image/art%3A10.1038%2Fs41380-020-00965-3/MediaObjects/41380_2020_965_Fig1_HTML.png Yes
https://www.frontiersin.org/files/Articles/867825/fnins-16-867825-HTML-r1/image_m/fnins-16-867825-g002.jpg Yes
https://pub.mdpi-res.com/brainsci/brainsci-10-00852/article_deploy/html/images/brainsci-10-00852-g002.png?1607087879 Yes
https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41582-021-00593-7/MediaObjects/41582_2021

https://media.springernature.com/w215h120/springer-static/image/art%3A10.1007%2Fs12031-020-01767-6/MediaObjects/12031_2020_1767_Fig1_HTML.png Yes
https://journals.plos.org/plosbiology/article/figure/image?size=medium&id=10.1371/journal.pbio.3000970.g001 Yes
https://www.frontiersin.org/files/Articles/727060/fnins-15-727060-HTML-r1/image_m/fnins-15-727060-g001.jpg Yes
https://www.frontiersin.org/files/Articles/592214/fncel-14-592214-HTML/image_m/fncel-14-592214-g002.jpg Yes
https://pub.mdpi-res.com/cells/cells-11-01298/article_deploy/html/images/cells-11-01298-g001-550.jpg?1649760444 Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs00705-021-04959-6/MediaObjects/705_2021_4959_Fig1_HTML.jpg Yes
https://media.springernature.com/w215h120/springer-static/image/art%3A10.1007%2Fs00018-018-2854-4/MediaObjects/18_2018_2854_Fig1_HTML.png Yes
https://cdnintech.com/media/chapter/89142/1711007103/media/F7.png Yes
https://media.springernature.com/lw1200/springer-stati

https://media.springernature.com/w215h120/springer-static/image/art%3A10.1007%2Fs00705-021-04959-6/MediaObjects/705_2021_4959_Fig1_HTML.jpg Yes
https://media.springernature.com/w215h120/springer-static/image/art%3A10.1007%2Fs00705-023-05801-x/MediaObjects/705_2023_5801_Fig1_HTML.png Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs40035-022-00316-y/MediaObjects/40035_2022_316_Fig4_HTML.png Yes
https://www.bmj.com/content/bmj/382/bmj-2022-073923/F1.medium.jpg Yes
https://www.mdpi.com/brainsci/brainsci-12-00190/article_deploy/html/images/brainsci-12-00190-g001.png Yes
https://www.frontiersin.org/files/Articles/1347987/fnagi-16-1347987-HTML/image_m/fnagi-16-1347987-g002.jpg Yes
https://images-provider.frontiersin.org/api/ipx/w=1200&f=png/https://www.frontiersin.org/files/Articles/572090/fnagi-12-572090-HTML/image_m/fnagi-12-572090-g001.jpg Yes
https://www.frontiersin.org/files/Articles/1249282/fnins-17-1249282-HTML/image_m/fnins-17-1249282-g007.jpg Yes
htt

https://media.springernature.com/lw1200/springer-static/image/art%3A10.1007%2Fs00702-022-02500-w/MediaObjects/702_2022_2500_Fig2_HTML.png Yes
https://www.mdpi.com/brainsci/brainsci-10-00852/article_deploy/html/images/brainsci-10-00852-g005.png Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs12974-022-02579-8/MediaObjects/12974_2022_2579_Fig3_HTML.png Yes
https://cdnintech.com/media/chapter/84633/1512345123/media/F1.png Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs12640-020-00219-8/MediaObjects/12640_2020_219_Fig1_HTML.png Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs42399-020-00522-7/MediaObjects/42399_2020_522_Fig1b_HTML.png Yes
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1038%2Fs41583-023-00769-8/MediaObjects/41583_2023_769_Fig3_HTML.png Yes
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1007%2Fs13365-021-01014-7/MediaObjects/1

https://media.springernature.com/lw1200/springer-static/image/art%3A10.1007%2Fs11302-024-09998-7/MediaObjects/11302_2024_9998_Fig1_HTML.png Yes
https://assets.newatlas.com/dims4/default/b8c091e/2147483647/strip/true/crop/6000x4000+0+0/resize/2880x1920!/quality/90/?url=http%3A%2F%2Fnewatlas-brightspot.s3.amazonaws.com%2F24%2F79%2Fae1fd3864aed9203775c53438959%2Fdepositphotos-481532886-xl.jpg No
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1186%2Fs12974-020-01896-0/MediaObjects/12974_2020_1896_Fig1_HTML.png Yes
https://media.springernature.com/w215h120/springer-static/image/art%3A10.1038%2Fs41420-024-01915-6/MediaObjects/41420_2024_1915_Fig1_HTML.png Yes
https://www.frontiersin.org/files/Articles/660087/fneur-12-660087-HTML/image_m/fneur-12-660087-g002.jpg Yes
https://www.frontiersin.org/files/Articles/940969/fimmu-13-940969-HTML-r1/image_m/fimmu-13-940969-g001.jpg Yes
https://www.frontiersin.org/files/Articles/1227896/fnmol-16-1227896-HTML/image_m/fnmol-16-12278

https://pbs.twimg.com/media/FhJ7kB7UUAALMNg?format=jpg&name=4096x4096 Yes
https://www.aging-us.com/special-collection-cover/covid-19-part-2/large Yes
https://www.frontiersin.org/files/Articles/1146704/fimmu-14-1146704-HTML/image_m/fimmu-14-1146704-g001.jpg Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1186%2Fs12979-020-00196-8/MediaObjects/12979_2020_196_Fig1_HTML.png Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs12016-021-08909-7/MediaObjects/12016_2021_8909_Fig1_HTML.png Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs00281-020-00824-x/MediaObjects/281_2020_824_Fig1_HTML.png Yes
https://www.mdpi.com/ijms/ijms-24-12648/article_deploy/html/images/ijms-24-12648-g006-550.jpg Yes
https://www.frontiersin.org/files/Articles/553331/fneur-11-00498-HTML/image_m/fneur-11-00498-g001.jpg Yes
https://minio.scielo.br/documentstore/1809-452X/tyQfYHcGcnLxrhhcy5gyZ3L/0d229539b970a8f5980791dfbf677866b0efce

https://www.frontiersin.org/files/MyHome%20Article%20Library/836605/836605_Thumb_400.jpg Yes
https://www.frontiersin.org/files/Articles/691136/fncel-15-691136-HTML/image_m/fncel-15-691136-g001.jpg Yes
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1038%2Fs12276-022-00755-7/MediaObjects/12276_2022_755_Fig2_HTML.png Yes
https://cdnintech.com/media/chapter/64482/1512345123/media/F2.png No
https://www.h-h-c.com/wp-content/uploads/2024/02/Post-antioxidants.jpg Yes
https://www.suzannegazdamd.com/uploads/3/8/3/4/38341157/editor/blog-long-covid-3-brain.jpg?1661188307 Yes
https://content.iospress.com/media/jad/2023/96-4/jad-96-4-jad230396/jad-96-jad230396-g003.jpg Yes
https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41419-023-06102-6/MediaObjects/41419_2023_6102_Fig1_HTML.png Yes
https://uknow.uky.edu/sites/default/files/styles/uknow_story_image/public/COVID%20AD%20.jpg Yes
https://media.springernature.com/w215h120/springer-static/image/art%3A

https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-981-16-6703-9_3/MediaObjects/520252_1_En_3_Fig2_HTML.png Yes
https://media.springernature.com/w215h120/springer-static/image/art%3A10.1007%2Fs13205-024-03925-8/MediaObjects/13205_2024_3925_Fig1_HTML.png Yes
https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-3-031-18440-6_9/MediaObjects/495040_1_En_9_Fig1_HTML.png Yes
https://www.aginganddisease.org/article/2020/2152-5250/147903/null/thumbnail/ad-11-4-946-g1.jpg Yes
https://pub.mdpi-res.com/biomedicines/biomedicines-10-00250/article_deploy/html/images/biomedicines-10-00250-g010-550.jpg?1643028186 Yes
https://www.frontiersin.org/files/Articles/1233340/frdem-02-1233340-HTML-r1/image_m/frdem-02-1233340-g001.jpg Yes
https://www.mdpi.com/pharmaceuticals/pharmaceuticals-17-00788/article_deploy/html/images/pharmaceuticals-17-00788-g001.png Yes
https://media.springernature.com/lw1200/springer-static/image/art%3A10.1007%2Fs12035-023-037

https://pub.mdpi-res.com/ijms/ijms-25-07169/article_deploy/html/images/ijms-25-07169-ag.png?1720701703 Yes
https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fs41423-024-01181-7/MediaObjects/41423_2024_1181_Fig1_HTML.png Yes
https://pub.mdpi-res.com/cells/cells-09-01687/article_deploy/html/images/cells-09-01687-g001.png?1596116881 Yes
https://www.promegaconnections.com/wp-content/uploads/2021/01/61090669-Blood-Brain-Blog-image-1024x512-1.jpg Yes
https://www.immunopaedia.org.za/wp-content/uploads/2023/09/fx1.jpg Yes
Error processing (529, image_url        https://europepmc.org/articles/PMC7803673/bin/...
Relevance_GPT                                                  Yes
Name: 529, dtype: object): Error code: 400 - {'error': {'message': 'Error while downloading https://europepmc.org/articles/PMC7803673/bin/painreports-6-e885-g001.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}
https://media.springernature.com/lw1200/springer-st

https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs12035-020-02149-0/MediaObjects/12035_2020_2149_Fig3_HTML.png Yes
https://ve.scielo.org/img/revistas/ic/v64n1//0535-5133-ic-64-01-108-gf1.jpg Yes
https://europepmc.org/articles/PMC10310313/bin/nihms-1901771-f0005.jpg Yes
https://europepmc.org/articles/PMC9132392/bin/IANN_A_2076901_F0002_C.jpg Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1038%2Fs41398-021-01629-8/MediaObjects/41398_2021_1629_Fig2_HTML.png Yes
https://www.wsws.org/asset/6c871bf7-6b96-46ea-a291-0eda9cdce206?rendition=image1280 Yes
https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs12035-024-04421-z/MediaObjects/12035_2024_4421_Fig1_HTML.png Yes
https://www.mdpi.com/ijms/ijms-25-08715/article_deploy/html/images/ijms-25-08715-g002-550.jpg Yes
https://www.mdpi.com/ijms/ijms-25-08715/article_deploy/html/images/ijms-25-08715-g003-550.jpg Yes
https://media.springernature.com/lw1200/springer-static

In [84]:
# Create a new DataFrame to store parsed results

# Loop through the list and clean up 'Yes.' or 'No.' to 'Yes' or 'No'
for item in parsed_data_2nd:
    # Check if the second element ends with a period and is 'Yes.' or 'No.'
    if item[1] in ['Yes.', 'No.']:
        item[1] = item[1].rstrip('.')

relevance_GPT_2nd = pd.DataFrame(parsed_data_2nd, columns=['URL', 'Relevance_GPT'])
#relevance_GPT_2nd.to_excel(r"Relevance_GPT_analysis_2nd.xlsx", index=False)
relevance_GPT_2nd

,URL,Relevance_GPT
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.frontiersin.org/files/Articles/103...,Yes
2,https://www.frontiersin.org/files/Articles/583...,Yes
3,https://media.springernature.com/lw1200/spring...,Yes
4,https://media.springernature.com/w215h120/spri...,Yes
...,...,...
621,https://i.ytimg.com/vi/fs3wDh_7pPE/hq720.jpg?s...,No
622,https://media.springernature.com/lw685/springe...,Yes
623,https://media.springernature.com/lw685/springe...,No
624,https://www.aging-us.com/article/202136/figure...,Yes


In [89]:
print('Number of relevant images (GPT):', relevance_GPT_2nd['Relevance_GPT'].value_counts().get('Yes', 0))

Number of relevant images (GPT): 567


In [96]:
Main_list_URLs_manual = relevance_GPT_2nd[relevance_GPT_2nd['Relevance_GPT'] == 'Yes']
Main_list_URLs_manual.reset_index(drop=True, inplace=True)
Main_list_URLs_manual

,URL,Relevance_GPT
0,https://media.springernature.com/lw1200/spring...,Yes
1,https://www.frontiersin.org/files/Articles/103...,Yes
2,https://www.frontiersin.org/files/Articles/583...,Yes
3,https://media.springernature.com/lw1200/spring...,Yes
4,https://media.springernature.com/w215h120/spri...,Yes
...,...,...
562,https://media.springernature.com/lw685/springe...,Yes
563,https://europepmc.org/articles/PMC5319859/bin/...,Yes
564,https://media.licdn.com/dms/image/C4D12AQGOchn...,Yes
565,https://media.springernature.com/lw685/springe...,Yes


In [107]:
# Save URL list for the final manual analysis
#(Main_list_URLs_manual[['URL']]).to_excel('C:\\Users\\User\\Desktop\\Data\\SCAI\\Image_analysis_AI\\Main_list_URLs_manual.xlsx', index=False)

# Open manually analyzed URL list
URLs_manual_analyzed = pd.read_excel(path_1+"URL_manual_main_checked.xlsx")
FINAL_RELEVANT_URLs = URLs_manual_analyzed[URLs_manual_analyzed['Relevance_manual'] == 'Yes']
FINAL_RELEVANT_URLs.reset_index(drop=True, inplace=True)
FINAL_RELEVANT_URLs

,URL,Relevance_manual
0,https://media.springernature.com/lw685/springe...,Yes
1,https://media.springernature.com/lw1200/spring...,Yes
2,https://www.eurekaselect.com/images/graphical-...,Yes
3,https://media.springernature.com/lw685/springe...,Yes
4,https://qeios-uploads.s3.eu-west-1.amazonaws.c...,Yes
...,...,...
284,https://f6publishing.blob.core.windows.net/728...,Yes
285,https://europepmc.org/articles/PMC9922164/bin/...,Yes
286,https://www.mdpi.com/cells/cells-12-00816/arti...,Yes
287,https://media.springernature.com/lw685/springe...,Yes


In [108]:
# Save the final file
FINAL_RELEVANT_URLs.to_excel(path_1+"FINAL_RELEVANT_URLs.xlsx")